In [ ]:
import matplotlib
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import os
import umap
import datashader as ds
import colorcet as cc
import igraph
import tqdm
import pandas as pd
from sklearn.neighbors import NearestNeighbors

from dredFISH.Analysis import TissueGraph
from dredFISH.Visualization import Viz
from dredFISH.Utils.__init__plots import * 
from dredFISH.Utils import powerplots
import importlib
importlib.reload(Viz)
importlib.reload(TissueGraph)

#### Load data

In [ ]:
respath = '/bigstore/GeneralStorage/fangming/projects/dredfish/figures/'

In [ ]:
basepth = '/bigstore/GeneralStorage/Data/dredFISH/Dataset1'
!ls -alhtr $basepth
!head $basepth"/TMG.json"

In [ ]:
df = pd.read_csv(
    os.path.join(basepth, "analysis_dev.csv"))
df

In [ ]:
TMG = TissueGraph.TissueMultiGraph(basepath=basepth, 
                                   redo=False, # load existing 
                                  )
TMG

In [ ]:
# spatial coordinates
layer = TMG.Layers[0]
XY = layer.XY
x, y = XY[:,0], XY[:,1]
###
x, y = y, x # a temporary hack
###

N = layer.N
print(N)
# measured basis
ftrs_mat = TMG.Layers[0].feature_mat

# umap_mat = umap.UMAP(n_neighbors=30, min_dist=0.1).fit_transform(ftrs_mat)

# types

# regions

# Local env (Isozones)
- same cell type zones
- instead quantify the nearest neighbors of the same cell type

In [ ]:
# call isozones
# compare the properties of isozone for each cell type
hue = 'type_r0.1'
labels = df[hue].values
labels

In [ ]:
# isozone
EL = np.asarray(layer.SG.get_edgelist())
EL_types = labels[EL]
EL_sametypes = EL[EL_types[:,0] == EL_types[:,1]]

IsoZoneG = igraph.Graph(n=N, edges=EL_sametypes, directed=False)
IsoZOneG = IsoZoneG.as_undirected().simplify() # what this means?
cmp = IsoZoneG.components()
Ix = np.asarray(cmp.membership)

isozones = Ix

In [ ]:
labels, isozones

In [ ]:
# Ix.shape
unq, cnts = np.unique(Ix, return_counts=True)
plt.plot(np.log10(np.sort(cnts)))

In [ ]:

uniqlabels = np.sort(np.unique(labels))
for i in uniqlabels:
    subIso = isozones[labels==i]
    unq, cnts = np.unique(subIso, return_counts=True)
    # plt.plot(np.log10(np.sort(cnts)), np.linspace(0, 1, len(cnts)), label=i)
    _x = np.cumsum(np.sort(cnts))
    plt.plot(np.log10(np.sort(cnts)), _x/_x[-1], label=i)
    # plt.plot(np.sort(cnts), np.linspace(0, 1, len(cnts)), label=i)
plt.legend()
plt.xlabel('log10(isozone size)')
plt.ylabel('CDF - Num. Isozones or Cells')
    

## Spatial nearest neighbors 
- distance to the nearest neighbors
- dependencies between cell types?


- the critical scale at which cells talk to each other?

In [ ]:
%%time
# slow for 24 bits; fast for 2D
k = 30 
NN = NearestNeighbors(n_neighbors=k)
NN.fit(XY)
knn = NN.kneighbors(XY, return_distance=False)

knn
# use pynndescent

In [ ]:
fracs = (labels.reshape(-1,1) == labels[knn[:,1:]]).sum(axis=1)/k

In [ ]:
x = df['x'].values
y = df['y'].values
knn1 = knn[:,1]
dists = np.sqrt(np.power(x[knn1] - x, 2) + np.power(y[knn1] - y, 2))
dists


In [ ]:
uniqlabels = np.sort(np.unique(labels))
for i in uniqlabels:
    sub_dists = dists[labels==i]
    plt.plot(np.sort(sub_dists), np.linspace(0, 1, len(sub_dists)), label=i)
plt.xlim([0,100])
plt.legend()
plt.xlabel('Distance (um) to nearest neighbor (regardless of cell types)')
plt.ylabel('CDF - Num. cells')
    

In [ ]:
uniqlabels = np.sort(np.unique(labels))
for i in uniqlabels:
    sub_fracs = fracs[labels==i]
    plt.plot(np.sort(sub_fracs), np.linspace(0, 1, len(sub_fracs)), label=i)
# plt.xlim([0,100])
plt.legend(bbox_to_anchor=(1,1))
plt.xlabel('Fraction of kNNs (k=30) of the same cell types')
plt.ylabel('CDF - Num. cells')
    

# Local env (Brain regions)